In [4]:
import re
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
driver.get("https://mobelix.com.mk/mk")
wait = WebDriverWait(driver, 10)

# чекање се додека не се појави дропдаунот за карегориите
toggle = wait.until(EC.element_to_be_clickable((By.ID, "navbarDropdown")))

# ховер прави на категориите
actions = ActionChains(driver)
actions.move_to_element(toggle).perform()

# ако не помине ховер да прави клик
toggle.click()

# чека да се појави менито
links = wait.until(EC.visibility_of_all_elements_located((
    By.CSS_SELECTOR, "div.dropdown-menu a.dropdown-item"
)))

markets = {}
for link in links:
    href = link.get_attribute("href")
    path = urlparse(href).path.rstrip("/")
    raw_slug = path.split("/")[-1]
    key = raw_slug.replace("-", "_")
    markets[key] = href

driver.quit()

# зачувување во варијабли
url_mob_telefoni    = markets["mobilni_telefoni"]
url_mob_smartwatch  = markets["smart_chasovnici"]
url_mob_dodatoci    = markets["mobilni_dodatoci"]
url_mob_eksponati   = markets["eksponati"]

print("Телефони:",        url_mob_telefoni)
print("Smart часовници:", url_mob_smartwatch)
print("Додатоци:",        url_mob_dodatoci)
print("Експонати:",       url_mob_eksponati)


Телефони: https://mobelix.com.mk/mk/mobilni-telefoni
Smart часовници: https://mobelix.com.mk/mk/smart-chasovnici
Додатоци: https://mobelix.com.mk/mk/mobilni-dodatoci
Експонати: https://mobelix.com.mk/mk/eksponati


In [5]:
import re
import time
import json
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# конфигурација на паге параметар за изминување на страници
PAGE_PARAM   = "page"
OUTPUT_LIMIT = None

driver = webdriver.Chrome()
wait   = WebDriverWait(driver, 10)

def get_total_pages():
    pag_links = wait.until(EC.presence_of_all_elements_located((
        By.CSS_SELECTOR, "ul.pagination li a.page-link, ul.pagination li a"
    )))
    nums = [int(a.text) for a in pag_links if a.text.strip().isdigit()]
    return max(nums) if nums else 1

def scrape_listing_page(url):
    driver.get(url)
    # Скролање за сите продукти
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    
    wait.until(EC.presence_of_all_elements_located((
        By.CSS_SELECTOR, "div.p-2.rounded.text-dark.bg-white.d-flex.w-100"
    )))
    cards = driver.find_elements(By.CSS_SELECTOR,
        "div.p-2.rounded.text-dark.bg-white.d-flex.w-100")

    items = []
    for card in cards:
        driver.execute_script("arguments[0].scrollIntoView(true);", card)
        time.sleep(0.2)

        a = card.find_element(By.CSS_SELECTOR, "a.d-flex.w-100")
        href = a.get_attribute("href").strip()
        img  = card.find_element(By.TAG_NAME, "img")
        photo = img.get_attribute("src").strip()

        brand = card.find_element(By.CSS_SELECTOR, "h5").text.strip()
        title = card.find_element(By.CSS_SELECTOR, "h3").text.strip()

        try:
            orig = card.find_element(By.CSS_SELECTOR, "p.price del").text.strip()
        except:
            orig = None

        price_text = card.find_element(By.CSS_SELECTOR, "p.price")\
                         .get_attribute("innerText")
        current = re.sub(r".*?\n", "", price_text).strip()

        try:
            disc = card.find_element(By.CSS_SELECTOR, "span.badge").text.strip()
        except:
            disc = None

        items.append({
            "url":            href,
            "photo_url":      photo,
            "brand":          brand,
            "title":          title,
            "original_price": orig,
            "current_price":  current,
            "discount":       disc
        })

    return items

def main():
    driver.get(url_mob_eksponati)
    total_pages = get_total_pages()
    print(f"Found {total_pages} pages.")

    all_products = []
    for page in range(1, total_pages + 1):
        if OUTPUT_LIMIT and page > OUTPUT_LIMIT:
            break

        page_url = url_mob_eksponati if page == 1 else f"{url_mob_eksponati}?{PAGE_PARAM}={page}"
        print(f"> Scraping page {page}/{total_pages}")
        data = scrape_listing_page(page_url)
        print(f"  → {len(data)} products found.")
        all_products.extend(data)

        time.sleep(1)

    print(f"\nTotal products scraped: {len(all_products)}")

    df = pd.DataFrame(all_products)
    df = df[["url","photo_url","brand","title","original_price","current_price","discount"]]
    df.to_csv("mobelix_eksponati.csv", index=False, encoding="utf-8")
    print("→ CSV saved to mobelix_eksponati.csv")

if __name__ == "__main__":
    main()
    driver.quit()


Found 11 pages.
> Scraping page 1/11
  → 9 products found.
> Scraping page 2/11
  → 9 products found.
> Scraping page 3/11
  → 9 products found.
> Scraping page 4/11
  → 9 products found.
> Scraping page 5/11
  → 9 products found.
> Scraping page 6/11
  → 9 products found.
> Scraping page 7/11
  → 9 products found.
> Scraping page 8/11
  → 9 products found.
> Scraping page 9/11
  → 9 products found.
> Scraping page 10/11
  → 9 products found.
> Scraping page 11/11
  → 2 products found.

Total products scraped: 92
→ CSV saved to mobelix_eksponati.csv
